<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Working with Watson OpenScale - Headless Subscription

This notebook should be run using with **Python 3.9** runtime environment. 

It requires service credentials for the following services:
  * Watson OpenScale

  
The notebook will configure OpenScale with a Headless subscription

# Setup <a name="setup"></a>

## Package installation

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install --upgrade ibm-watson-machine-learning --user | tail -n 1
!pip install --upgrade ibm-watson-openscale --no-cache | tail -n 1

### Action: restart the kernel!

## Configure credentials

- WOS_CREDENTIALS (CP4D)

In [ ]:
#masked
WOS_CREDENTIALS = {
    "url": "<CP4D_URL>",
    "username": "<USERNAME>",
    "password": "<PASSWORD>",
    "version": "4.5"
}

WML_CREDENTIALS = WOS_CREDENTIALS.copy()
WML_CREDENTIALS["instance_id"] = "wml_local"

# Using the default OpenScale instance. Change the instance id, as needed.
SERVICE_INSTANCE_ID="00000000-0000-0000-0000-000000000000"

# Get training data statistics

In [ ]:
feature_columns=["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"]
cat_features=["CheckingStatus","CreditHistory","LoanPurpose","ExistingSavings","EmploymentDuration","Sex","OthersOnLoan","OwnsProperty","InstallmentPlans","Housing","Job","Telephone","ForeignWorker"]
class_label = "Risk"

### Get the training data

In [ ]:
!rm german_credit_data_biased_training.csv
!wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/german_credit_data_biased_training.csv

In [ ]:
import pandas as pd
data_df=pd.read_csv ("german_credit_data_biased_training.csv")
data_df.head()

### Generate the training data stats

In [ ]:
from ibm_watson_openscale.utils.training_stats import TrainingStats

In [ ]:
input_parameters = {
    "label_column": class_label,
    "feature_columns": feature_columns,
    "categorical_columns": cat_features,
    "fairness_inputs": None,  
    "problem_type" : "binary"
}

In [ ]:
training_stats = TrainingStats(data_df,input_parameters, explain=True, fairness=False, drop_na=True)

In [ ]:
config_json = training_stats.get_training_statistics()

In [ ]:
config_json["notebook_version"] = 5.0

In [ ]:
config_json

# Configure OpenScale 

The notebook will now import the necessary libraries and set up a Python OpenScale client.

In [ ]:
from ibm_watson_openscale import APIClient
from ibm_watson_openscale.utils import *
from ibm_watson_openscale.supporting_classes import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import *


import json
import requests
import base64
from requests.auth import HTTPBasicAuth
import time

## Get a instance of the OpenScale SDK client

In [ ]:
authenticator = CloudPakForDataAuthenticator(
        url=WOS_CREDENTIALS['url'],
        username=WOS_CREDENTIALS['username'],
        password=WOS_CREDENTIALS['password'],
        disable_ssl_verification=True, 
    )

wos_client = APIClient(service_url=WOS_CREDENTIALS['url'],authenticator=authenticator)
wos_client.version

## Set up datamart

Watson OpenScale uses a database to store payload logs and calculated metrics. If database credentials were not supplied above, the notebook will use the free, internal lite database. If database credentials were supplied, the datamart will be created there unless there is an existing datamart and the KEEP_MY_INTERNAL_POSTGRES variable is set to True. If an OpenScale datamart exists in Db2 or PostgreSQL, the existing datamart will be used and no data will be overwritten.

Prior instances of the model will be removed from OpenScale monitoring.

In [ ]:
wos_client.data_marts.show()

In [ ]:
data_marts = wos_client.data_marts.list().result.data_marts
data_mart_id=data_marts[0].metadata.id
print('Using existing datamart {}'.format(data_mart_id))

In [ ]:
data_mart_details = wos_client.data_marts.list().result.data_marts[0]
data_mart_details.to_dict()

In [ ]:
wos_client.service_providers.show()

## Remove existing service provider

Multiple service providers for the same engine instance are avaiable in Watson OpenScale. To avoid multiple service providers of used WML instance in the tutorial notebook the following code deletes existing service provder(s) and then adds new one.

In [ ]:
SERVICE_PROVIDER_NAME = "RC - OpenScale Headless Service Provider"
SERVICE_PROVIDER_DESCRIPTION = "Added by tutorial WOS notebook to showcase Headless Subscription functionality."

In [ ]:
service_providers = wos_client.service_providers.list().result.service_providers
for service_provider in service_providers:
    service_instance_name = service_provider.entity.name
    if service_instance_name == SERVICE_PROVIDER_NAME:
        service_provider_id = service_provider.metadata.id
        wos_client.service_providers.delete(service_provider_id)
        print("Deleted existing service_provider for WML instance: {}".format(service_provider_id))

## Add service provider

Watson OpenScale needs to be bound to the Watson Machine Learning instance to capture payload data into and out of the model.

Note: Here the service provider is created with empty credentials, meaning no endpoint. Just to demonstrate the use case were we don't need an actual end point serving requests.

In [ ]:
MLCredentials = {}
added_service_provider_result = wos_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description=SERVICE_PROVIDER_DESCRIPTION,
        service_type=ServiceTypes.CUSTOM_MACHINE_LEARNING,
        operational_space_id = "production",
        credentials=MLCredentials,
        background_mode=False
    ).result
service_provider_id = added_service_provider_result.metadata.id

In [ ]:
print(wos_client.service_providers.get(service_provider_id).result)

## Subscriptions

Remove existing credit risk subscriptions

This code removes previous subscriptions to the model to refresh the monitors with the new model and new data.

In [ ]:
wos_client.subscriptions.show()

## Remove the existing subscription

In [ ]:
SUBSCRIPTION_NAME = "RC - GCR Headless Subscription"

In [ ]:
subscriptions = wos_client.subscriptions.list().result.subscriptions
for subscription in subscriptions:
    if subscription.entity.asset.name == '[asset] ' + SUBSCRIPTION_NAME:
        sub_model_id = subscription.metadata.id
        wos_client.subscriptions.delete(subscription.metadata.id)
        print('Deleted existing subscription for model', sub_model_id)

This code creates the model subscription in OpenScale using the Python client API. Note that we need to provide the model unique identifier, and some information about the model itself.

In [ ]:
print("Data Mart ID: " + data_mart_id)
print("Service Provide ID: " + service_provider_id)
import uuid
asset_id = str(uuid.uuid4())
asset_name = '[asset] ' + SUBSCRIPTION_NAME
url = ''

asset_deployment_id = str(uuid.uuid4())
asset_deployment_name = asset_name

In [ ]:
prediction_column = "prediction"
probability_columns = ['probability']
predicted_target_column = "prediction"
subscription_details = wos_client.subscriptions.add(data_mart_id,
    service_provider_id,
    asset=Asset(
        asset_id=asset_id,
        name=asset_name,
        url=url,
        asset_type=AssetTypes.MODEL,
        input_data_type=InputDataType.STRUCTURED,
        problem_type=ProblemType.BINARY_CLASSIFICATION
    ),
    deployment=AssetDeploymentRequest(
        deployment_id=asset_deployment_id,
        name=asset_deployment_name,
        deployment_type= DeploymentTypes.ONLINE
    ),    
    training_data_stats=config_json,
    prediction_field = prediction_column,
    predicted_target_field = predicted_target_column,
    probability_fields = probability_columns,background_mode = False
    ,deployment_name = asset_name
    ).result

subscription_id = subscription_details.metadata.id
print("Subscription id {}".format(subscription_id))

In [ ]:
import time

time.sleep(5)
payload_data_set_id = None
payload_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
if payload_data_set_id is None:
    print("Payload data set not found. Please check subscription status.")
else:
    print("Payload data set id:", payload_data_set_id)

In [ ]:
wos_client.subscriptions.get(subscription_id).result.to_dict()

<!-- ## Push a payload record to setup the required schemas in the subscription

Note : No scoring is done against the model. The PayloadRecord is constructed with the request and response from the model/deployment. -->

## Push a payload record to setup the required schemas in the subscription

Note : No scoring is done against the model. The PayloadRecord is constructed with the request and response from the model/deployment.

## Scoring Request Payload

In [ ]:
scoring_request =   {
        "fields": [
            "CheckingStatus",
            "LoanDuration",
            "CreditHistory",
            "LoanPurpose",
            "LoanAmount",
            "ExistingSavings",
            "EmploymentDuration",
            "InstallmentPercent",
            "Sex",
            "OthersOnLoan",
            "CurrentResidenceDuration",
            "OwnsProperty",
            "Age",
            "InstallmentPlans",
            "Housing",
            "ExistingCreditsCount",
            "Job",
            "Dependents",
            "Telephone",
            "ForeignWorker",
            "Risk"
        ],
        "values": [
            [
                "no_checking",
                28,
                "outstanding_credit",
                "appliances",
                5990,
                "500_to_1000",
                "greater_7",
                5,
                "male",
                "co-applicant",
                3,
                "car_other",
                55,
                "none",
                "free",
                2,
                "skilled",
                2,
                "yes",
                "yes",
                "Risk"
            ],
            [
                "greater_200",
                22,
                "all_credits_paid_back",
                "car_used",
                3376,
                "less_100",
                "less_1",
                3,
                "female",
                "none",
                2,
                "car_other",
                32,
                "none",
                "own",
                1,
                "skilled",
                1,
                "none",
                "yes",
                "No Risk"
            ],
            [
                "no_checking",
                39,
                "credits_paid_to_date",
                "vacation",
                6434,
                "unknown",
                "greater_7",
                5,
                "male",
                "none",
                4,
                "car_other",
                39,
                "none",
                "own",
                2,
                "skilled",
                2,
                "yes",
                "yes",
                "Risk"
            ],
            [
                "0_to_200",
                20,
                "credits_paid_to_date",
                "furniture",
                2442,
                "less_100",
                "unemployed",
                3,
                "female",
                "none",
                1,
                "real_estate",
                42,
                "none",
                "own",
                1,
                "skilled",
                1,
                "none",
                "yes",
                "No Risk"
            ],
            [
                "greater_200",
                4,
                "all_credits_paid_back",
                "education",
                4206,
                "less_100",
                "unemployed",
                1,
                "female",
                "none",
                3,
                "savings_insurance",
                27,
                "none",
                "own",
                1,
                "management_self-employed",
                1,
                "none",
                "yes",
                "No Risk"
            ],
            [
                "greater_200",
                23,
                "credits_paid_to_date",
                "car_used",
                2963,
                "greater_1000",
                "greater_7",
                4,
                "male",
                "none",
                4,
                "car_other",
                46,
                "none",
                "own",
                2,
                "skilled",
                1,
                "none",
                "yes",
                "Risk"
            ],
            [
                "no_checking",
                31,
                "prior_payments_delayed",
                "vacation",
                2673,
                "500_to_1000",
                "1_to_4",
                3,
                "male",
                "none",
                2,
                "real_estate",
                35,
                "stores",
                "rent",
                1,
                "skilled",
                2,
                "none",
                "yes",
                "Risk"
            ],
            [
                "no_checking",
                37,
                "prior_payments_delayed",
                "other",
                6971,
                "500_to_1000",
                "1_to_4",
                3,
                "male",
                "none",
                3,
                "savings_insurance",
                54,
                "none",
                "own",
                2,
                "skilled",
                1,
                "yes",
                "yes",
                "Risk"
            ],
            [
                "no_checking",
                14,
                "all_credits_paid_back",
                "car_new",
                1525,
                "500_to_1000",
                "4_to_7",
                3,
                "male",
                "none",
                4,
                "real_estate",
                33,
                "none",
                "own",
                1,
                "skilled",
                1,
                "none",
                "yes",
                "No Risk"
            ],
            [
                "less_0",
                10,
                "prior_payments_delayed",
                "furniture",
                4037,
                "less_100",
                "4_to_7",
                3,
                "male",
                "none",
                3,
                "savings_insurance",
                31,
                "none",
                "rent",
                1,
                "skilled",
                1,
                "none",
                "yes",
                "Risk"
            ]
        ],
        "meta": {
            "fields": [
                "referrer_gender"
            ],
            "values": [
                [
                    "male"
                ],
                [
                    "female"
                ],
                [
                    "male"
                ],
                [
                    "female"
                ],
                [
                    "female"
                ],
                [
                    "male"
                ],
                [
                    "female"
                ],
                [
                    "male"
                ],
                [
                    "female"
                ],
                [
                    "male"
                ]
            ]
        }
    }

## Scoring Response Payload

In [ ]:
scoring_response = {
    "predictions": [
        {
            "fields": [
                "prediction",
                "probability"
            ],
            "values": [
                [
                    "Risk",
                    [
                        0.104642951112211,
                        0.895357048887789
                    ]
                ],
                [
                    "No Risk",
                    [
                        0.892112895920181,
                        0.10788710407981907
                    ]
                ],
                [
                    "Risk",
                    [
                        0.4863177905287259,
                        0.5136822094712741
                    ]
                ],
                [
                    "No Risk",
                    [
                        0.980811537315731,
                        0.01918846268426898
                    ]
                ],
                [
                    "No Risk",
                    [
                        0.9053052561083984,
                        0.09469474389160164
                    ]
                ],
                [
                    "No Risk",
                    [
                        0.5315146773053994,
                        0.4684853226946007
                    ]
                ],
                [
                    "No Risk",
                    [
                        0.7689466209701616,
                        0.23105337902983833
                    ]
                ],
                [
                    "Risk",
                    [
                        0.41317664143643873,
                        0.5868233585635613
                    ]
                ],
                [
                    "No Risk",
                    [
                        0.9190247585206522,
                        0.08097524147934775
                    ]
                ],
                [
                    "No Risk",
                    [
                        0.781841942776921,
                        0.21815805722307902
                    ]
                ]
            ]
        }
    ]
}

In [ ]:
from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord

records_list=[]
for x in range(10):
    pl_record = PayloadRecord(request=scoring_request, response=scoring_response)
    records_list.append(pl_record)

wos_client.data_sets.store_records(data_set_id=payload_data_set_id, request_body=records_list)

In [ ]:
time.sleep(5)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))
if pl_records_count == 0:
    raise Exception("Payload logging did not happen!")

## Fetch the subscription details to confirm schemas are setup

In [ ]:
wos_client.subscriptions.get(subscription_id).result.to_dict()

# Explainability Monitor Configuration

In [ ]:
import urllib3, requests, json
def generate_access_token():
    headers={}
    headers["Accept"] = "application/json"
    auth = HTTPBasicAuth(WOS_CREDENTIALS["username"], WOS_CREDENTIALS["password"])
    
    ICP_TOKEN_URL= WOS_CREDENTIALS["url"] + "/v1/preauth/validateAuth"
    
    response = requests.get(ICP_TOKEN_URL, headers=headers, auth=auth, verify=False)
    json_data = response.json()
    icp_access_token = json_data['accessToken']
    return icp_access_token

In [ ]:
icp_access_token = generate_access_token()

In [ ]:
header = {
    'Content-Type': 'application/json', 
    'Authorization': 'Bearer ' + icp_access_token
}

## Generate perturbations over the training stats

In [ ]:
from ibm_wos_utils.explainability.utils.perturbations import Perturbations   
perturbations=Perturbations(training_stats=config_json.get("explainability_configuration"), problem_type="binary", perturbations_count=5000)
perturbs_df = perturbations.generate_perturbations()

## Construct the scoring payload

In [ ]:
cols_to_remove = [class_label]
cols_to_remove

In [ ]:
def get_scoring_payload(no_of_records_to_score = 1):
    for col in cols_to_remove:
        if col in perturbs_df.columns:
            del perturbs_df[col] 

    fields = perturbs_df.columns.tolist()
    values = perturbs_df[fields].values.tolist()

    payload_scoring = {"input_data": [{"fields": fields, "values": values[:no_of_records_to_score]}]}  
    return payload_scoring

In [ ]:
def sample_scoring(no_of_records_to_score = 1):
    records_list=[]
    payload_scoring = get_scoring_payload(no_of_records_to_score)
    scoring_response = wml_client.deployments.score(deployment_uid, payload_scoring)
    print('Single record scoring result:', '\n fields:', scoring_response['predictions'][0]['fields'], '\n values: ', scoring_response['predictions'][0]['values'][0])
    #print(json.dumps(scoring_response, indent=None))
    return payload_scoring, scoring_response

## Score the perturbations

In [ ]:
import json
from ibm_watson_machine_learning import APIClient

wml_client = APIClient(WML_CREDENTIALS)
wml_client.version

In [ ]:
space_uid = ''
deployment_uid = ''

In [ ]:
wml_client.set.default_space(space_uid)

In [ ]:
payload_scoring, scoring_response = sample_scoring(no_of_records_to_score = 5000)

## Construct the explain archive

In [ ]:
fields = scoring_response['predictions'][0]['fields']
values = scoring_response['predictions'][0]['values']

In [ ]:
scored_data = pd.DataFrame(values, columns = fields)

In [ ]:
probabilities = [pro for pro in scored_data['probability']]
predictions = [pre for pre in scored_data['prediction']]

scored_perturbations = {'probabilities' : probabilities,
                        'predictions' : predictions}

In [ ]:
import tarfile
from io import BytesIO
from json import dumps

archive_data = {
        'lime_scored_perturbations.json': dumps(scored_perturbations),
        'training_statistics.json': dumps({"training_statistics": config_json.get("explainability_configuration")})
    }
filename = 'explainability.tar.gz'

with tarfile.open(filename, mode="w:gz") as tf:
    for filename, filedata in archive_data.items():
        content = BytesIO(filedata.encode("utf8"))
        tarinfo = tarfile.TarInfo(filename)
        tarinfo.size = len(content.getvalue())
        tf.addfile(
            tarinfo=tarinfo, fileobj=content)

## Upload the explain configuration archive to OpenScale data mart

In [ ]:
with open('explainability.tar.gz', mode='rb') as explainability_tar:
    wos_client.monitor_instances.upload_explainability_archive(subscription_id=subscription_id, archive=explainability_tar)

print('Uploaded explainability configuration archive successfully.')

## Enable explainability monitor

In [ ]:
print("Creating explainability monitor instance...")
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)
parameters={
    "enabled": True
}
response = wos_client.monitor_instances.create(monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID, 
                                               data_mart_id=data_mart_id,
                                               target=target,
                                               parameters=parameters,
                                               background_mode=False)
print(response)

## Trigger a local explanation

In [ ]:
payload_data = wos_client.data_sets.get_list_of_records(data_set_id=payload_data_set_id,output_type='pandas').result
explanation_types = ["lime"]

scoring_ids = payload_data.head(1)['scoring_id'].tolist()
result = wos_client.monitor_instances.explanation_tasks(scoring_ids=scoring_ids, explanation_types=explanation_types, subscription_id=subscription_id).result

explanation_task_ids=result.metadata.explanation_task_ids
explanation_task_ids

In [ ]:
def finish_explanation_tasks(sample_size = 1):
    finished_explanations = []
    finished_explanation_task_ids = []
    
    # Check for the explanation task status for finished status. 
    # If it is in-progress state, then sleep for some time and check again. 
    # Perform the same for couple of times, so that all tasks get into finished state.
    for i in range(0, 5):
        # for each explanation
        print('iteration ' + str(i))
        
        #check status for all explanation tasks
        for explanation_task_id in explanation_task_ids:
            if explanation_task_id not in finished_explanation_task_ids:
                result = wos_client.monitor_instances.get_explanation_tasks(explanation_task_id=explanation_task_id).result
                print(explanation_task_id + ' : ' + result.entity.status.state)
                if (result.entity.status.state == 'finished' or result.entity.status.state == 'error') and explanation_task_id not in finished_explanation_task_ids:
                    finished_explanation_task_ids.append(explanation_task_id)
                    finished_explanations.append(result)


        # if there is altest one explanation task that is not yet completed, then sleep for sometime, 
        # and check for all those tasks, for which explanation is not yet completeed.
        
        if len(finished_explanation_task_ids) != sample_size:
            print('sleeping for some time..')
            time.sleep(10)
        else:
            break
                    
    return finished_explanations

## Find the explain task status

In [ ]:
finished_explanations = finish_explanation_tasks(1)

## Print explain task output

In [ ]:
for result in finished_explanations:
    print(result)

# Fairness configuration

The code below configures fairness monitoring for our model. It turns on monitoring for two features, sex and age. In each case, we must specify:
    
Which model feature to monitor One or more majority groups, which are values of that feature that we expect to receive a higher percentage of favorable outcomes One or more minority groups, which are values of that feature that we expect to receive a higher percentage of unfavorable outcomes The threshold at which we would like OpenScale to display an alert if the fairness measurement falls below (in this case, 80%) Additionally, we must specify which outcomes from the model are favourable outcomes, and which are unfavourable. We must also provide the number of records OpenScale will use to calculate the fairness score. In this case, OpenScale's fairness monitor will run hourly, but will not calculate a new fairness rating until at least 100 records have been added. Finally, to calculate fairness, OpenScale must perform some calculations on the training data, so we provide the dataframe containing the data.

### Create Fairness Monitor Instance

In [ ]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id

)
parameters = {
    "features": [
        {"feature": "referrer_gender",
         "majority": ['female'],
         "minority": ['male'],
         "correlated_attributes": []
         }
    ],
    "favourable_class": ["No Risk"],
    "unfavourable_class": ["Risk"],
    "min_records": 95
}
thresholds = [{
    "metric_id": "fairness_value",
    "specific_values": [
        {
            "applies_to": [{
                "key": "feature",
                "type": "tag",
                "value": "referrer_gender"
            }],
            "value": 98
        }
    ],
    "type": "lower_limit",
    "value": 80.0
}]

fairness_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.FAIRNESS.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds).result

In [ ]:
fairness_monitor_instance_id = fairness_monitor_details.metadata.id

### Get Fairness Monitor Instance

In [ ]:
wos_client.monitor_instances.show()

### Get run details
In case of production subscription, initial monitoring run is triggered internally. Checking its status

In [ ]:
runs = wos_client.monitor_instances.list_runs(fairness_monitor_instance_id, limit=1).result.to_dict()
fairness_monitoring_run_id = runs["runs"][0]["metadata"]["id"]
run_status = None
while(run_status not in ["finished", "error"]):
    run_details = wos_client.monitor_instances.get_run_details(fairness_monitor_instance_id, fairness_monitoring_run_id).result.to_dict()
    run_status = run_details["entity"]["status"]["state"]
    print('run_status: ', run_status)
    if run_status in ["finished", "error"]:
        break
    time.sleep(10)

### Fairness run output

In [ ]:
wos_client.monitor_instances.get_run_details(fairness_monitor_instance_id, fairness_monitoring_run_id).result.to_dict()

In [ ]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=fairness_monitor_instance_id)

# Quality monitoring and feedback logging

## Enable quality monitoring

The code below waits ten seconds to allow the payload logging table to be set up before it begins enabling monitors. First, it turns on the quality (accuracy) monitor and sets an alert threshold of 70%. OpenScale will show an alert on the dashboard if the model accuracy measurement (area under the curve, in the case of a binary classifier) falls below this threshold.

The second paramater supplied, min_records, specifies the minimum number of feedback records OpenScale needs before it calculates a new measurement. The quality monitor runs hourly, but the accuracy reading in the dashboard will not change until an additional 50 feedback records have been added, via the user interface, the Python client, or the supplied feedback endpoint.

In [ ]:
import time

#time.sleep(10)
target = Target(
        target_type=TargetTypes.SUBSCRIPTION,
        target_id=subscription_id
)
parameters = {
    "min_feedback_data_size": 100
}
thresholds = [
                {
                    "metric_id": "area_under_roc",
                    "type": "lower_limit",
                    "value": .80
                }
            ]
quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result

In [ ]:
quality_monitor_instance_id = quality_monitor_details.metadata.id
quality_monitor_instance_id

## Get feedback logging dataset ID

In [ ]:
feedback_dataset_id = None
feedback_dataset = wos_client.data_sets.list(type=DataSetTypes.FEEDBACK, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result
feedback_dataset_id = feedback_dataset.data_sets[0].metadata.id
if feedback_dataset_id is None:
    print("Feedback data set not found. Please check quality monitor status.")

In [ ]:
feedback_dataset_id

In [ ]:
feedback_payload = {
    "fields": [
        "CheckingStatus",
        "LoanDuration",
        "CreditHistory",
        "LoanPurpose",
        "LoanAmount",
        "ExistingSavings",
        "EmploymentDuration",
        "InstallmentPercent",
        "Sex",
        "OthersOnLoan",
        "CurrentResidenceDuration",
        "OwnsProperty",
        "Age",
        "InstallmentPlans",
        "Housing",
        "ExistingCreditsCount",
        "Job",
        "Dependents",
        "Telephone",
        "ForeignWorker",
        "Risk",
        "_original_probability",
        "_original_prediction",
        "_debiased_probability",
        "_debiased_prediction"        
    ],
    "values": [
        [
            "less_0",
            18,
            "credits_paid_to_date",
            "car_new",
            462,
            "less_100",
            "1_to_4",
            2,
            "female",
            "none",
            2,
            "savings_insurance",
            37,
            "stores",
            "own",
            2,
            "skilled",
            1,
            "none",
            "yes",
            "No Risk",
            [
                0.767955712021837,
                0.23204428797816307
            ],
            "Risk",
            [
                0.767955712021837,
                0.23204428797816307
            ],
            "Risk"
        ],
        [
            "less_0",
            15,
            "prior_payments_delayed",
            "furniture",
            250,
            "less_100",
            "1_to_4",
            2,
            "male",
            "none",
            3,
            "real_estate",
            28,
            "none",
            "own",
            2,
            "skilled",
            1,
            "yes",
            "no",
            "No Risk",
            [
                0.7419002139563244,
                0.25809978604367556
            ],
            "Risk",
            [
                0.767955712021837,
                0.23204428797816307
            ],
            "Risk"
        ],
        [
            "0_to_200",
            28,
            "credits_paid_to_date",
            "retraining",
            3693,
            "less_100",
            "greater_7",
            3,
            "male",
            "none",
            2,
            "savings_insurance",
            32,
            "none",
            "own",
            1,
            "skilled",
            1,
            "none",
            "yes",
            "No Risk",
            [
                0.6935080115729353,
                0.3064919884270647
            ],
            "Risk",
            [
                0.8,
                0.2
            ],
            "Risk"
        ],
        [
            "no_checking",
            28,
            "prior_payments_delayed",
            "education",
            6235,
            "500_to_1000",
            "greater_7",
            3,
            "male",
            "none",
            3,
            "unknown",
            57,
            "none",
            "own",
            2,
            "skilled",
            1,
            "none",
            "yes",
            "Risk",
            [
                0.331110352092386,
                0.668889647907614
            ],
            "Risk",
            [
                0.9,
                0.1
            ],
            "Risk"
        ],
        [
            "no_checking",
            32,
            "outstanding_credit",
            "vacation",
            9604,
            "500_to_1000",
            "greater_7",
            6,
            "male",
            "co-applicant",
            5,
            "unknown",
            57,
            "none",
            "free",
            2,
            "skilled",
            2,
            "yes",
            "yes",
            "Risk",
            [
                0.11270206970758759,
                0.8872979302924124
            ],
            "Risk",
            [
                0.1,
                0.9
            ],
            "Risk"
        ],
        [
            "no_checking",
            9,
            "prior_payments_delayed",
            "car_new",
            1032,
            "100_to_500",
            "4_to_7",
            3,
            "male",
            "none",
            4,
            "savings_insurance",
            41,
            "none",
            "own",
            1,
            "management_self-employed",
            1,
            "none",
            "yes",
            "No Risk",
            [
                0.6704819620865308,
                0.32951803791346923
            ],
            "Risk",
            [
                0.767955712021837,
                0.23204428797816307
            ],
            "Risk"
        ],
        [
            "less_0",
            16,
            "credits_paid_to_date",
            "vacation",
            3109,
            "less_100",
            "4_to_7",
            3,
            "female",
            "none",
            1,
            "car_other",
            36,
            "none",
            "own",
            2,
            "skilled",
            1,
            "none",
            "yes",
            "No Risk",
            [
                0.6735810290914039,
                0.3264189709085961
            ],
            "Risk",
            [
                0.6,
                0.4
            ],
            "Risk"
        ],
        [
            "0_to_200",
            11,
            "credits_paid_to_date",
            "car_new",
            4553,
            "less_100",
            "less_1",
            3,
            "female",
            "none",
            3,
            "savings_insurance",
            22,
            "none",
            "own",
            1,
            "management_self-employed",
            1,
            "none",
            "yes",
            "No Risk",
            [
                0.637964656269084,
                0.362035343730916
            ],
            "Risk",
            [
                0.767955712021837,
                0.23204428797816307
            ],
            "Risk"
        ],
        [
            "no_checking",
            35,
            "outstanding_credit",
            "appliances",
            7138,
            "500_to_1000",
            "greater_7",
            5,
            "male",
            "co-applicant",
            4,
            "unknown",
            49,
            "none",
            "free",
            2,
            "skilled",
            2,
            "yes",
            "yes",
            "Risk",
            [
                0.11270206970758759,
                0.8872979302924124
            ],
            "Risk",
            [
                0.767955712021837,
                0.23204428797816307
            ],
            "Risk"
        ],
        [
            "less_0",
            5,
            "all_credits_paid_back",
            "car_new",
            1523,
            "less_100",
            "unemployed",
            2,
            "female",
            "none",
            2,
            "real_estate",
            19,
            "none",
            "rent",
            1,
            "management_self-employed",
            1,
            "none",
            "yes",
            "No Risk",
            [
                0.7304597628653227,
                0.26954023713467745
            ],
            "Risk",
            [
                0.767955712021837,
                0.23204428797816307
            ],
            "Risk"
        ]
    ]
}

### Store the feedback payload using the data sets API

In [ ]:
DATASETS_STORE_RECORDS_URL =   WOS_CREDENTIALS["url"] + "/openscale/{0}/v2/data_sets/{1}/records".format(data_mart_id, feedback_dataset_id)
for x in range(10):
    response = requests.post(DATASETS_STORE_RECORDS_URL, json=feedback_payload, headers=header, verify=False)
    json_data = response.json()
    print(json_data)

### Wait for sometime, and make sure the records have reached to data sets related table.

In [ ]:
time.sleep(10)
DATASETS_STORE_RECORDS_URL =   WOS_CREDENTIALS["url"] + "/openscale/{0}/v2/data_sets/{1}/records?limit={2}&include_total_count={3}".format(data_mart_id, feedback_dataset_id, 1, "true")
response = requests.get(DATASETS_STORE_RECORDS_URL, headers=header, verify=False)
json_data = response.json()
print(json_data['total_count'])

## Run Quality Monitor

In [ ]:
run_details = wos_client.monitor_instances.run(monitor_instance_id=quality_monitor_instance_id, background_mode=False).result

In [ ]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=quality_monitor_instance_id)

# Drift Configuration

## Create the drift detection model archive

In [ ]:
def score(training_data_frame):
      
    #The data type of the label column and prediction column should be same .
    #User needs to make sure that label column and prediction column array should have the same unique class labels
    prediction_column_name = "predictedLabel"
    probability_column_name = "probability"
        
    feature_columns = list(training_data_frame.columns)
    training_data_rows = training_data_frame[feature_columns].values.tolist()
    
    payload_scoring = {
      wml_client.deployments.ScoringMetaNames.INPUT_DATA: [{
           "fields": feature_columns,
           "values": [x for x in training_data_rows]
      }]
    }

    score = wml_client.deployments.score(deployment_uid, payload_scoring)
    score_predictions = score.get('predictions')[0]

    prob_col_index = list(score_predictions.get('fields')).index(probability_column_name)
    predict_col_index = list(score_predictions.get('fields')).index(prediction_column_name)

    if prob_col_index < 0 or predict_col_index < 0:
        raise Exception("Missing prediction/probability column in the scoring response")

    import numpy as np
    probability_array = np.array([value[prob_col_index] for value in score_predictions.get('values')])
    prediction_vector = np.array([value[predict_col_index] for value in score_predictions.get('values')])

    return probability_array, prediction_vector

In [ ]:
import pandas as pd
training_data_frame=pd.read_csv ("german_credit_data_biased_training.csv")
training_data_frame.head()

In [ ]:
probability_array, prediction_vector = score(training_data_frame)
prediction_vector

In [ ]:
from ibm_wos_utils.drift.drift_trainer import DriftTrainer

drift_detection_input = {
    "feature_columns": feature_columns,
    "categorical_columns":cat_features,
    "label_column": class_label,
    "problem_type": "binary"
}

drift_trainer = DriftTrainer(training_data_frame, drift_detection_input)
drift_trainer.generate_drift_detection_model(score, batch_size=training_data_frame.shape[0], check_for_ddm_quality=False)
drift_trainer.learn_constraints(
    two_column_learner_limit=200, categorical_unique_threshold=0.8, user_overrides=[])
drift_trainer.create_archive()

### Enable the drift monitor

In the following code cell, type a path to the drift configuration tar ball.

In [ ]:
wos_client.monitor_instances.upload_drift_model(
    model_path="drift_detection_model.tar.gz",
    data_mart_id=data_mart_id,
    subscription_id=subscription_id
).result

In the following code cell, default values are set for the drift monitor. You can change the default values by updating the values in the parameters section. The min_samples parameter controls the number of records that triggers the drift monitor to run. The drift_threshold parameter sets the threshold in decimal format for the drift percentage to trigger an alert. The train_drift_model parameter controls whether to re-train the model based on the drift analysis.

In [ ]:
import time

target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = {
    "min_samples": 100,
    "drift_threshold": 0.05,
    "train_drift_model": False
}

drift_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT.ID,
    target=target,
    parameters=parameters
).result

drift_monitor_instance_id = drift_monitor_details.metadata.id
print(drift_monitor_details)

## Check monitor instance status

In [ ]:
drift_status = None

while drift_status not in ("active", "error"):
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=drift_monitor_instance_id).result
    drift_status = monitor_instance_details.entity.status.state
    if drift_status not in ("active", "error"):
        print(datetime.utcnow().strftime('%H:%M:%S'), drift_status)
        time.sleep(30)

print(datetime.utcnow().strftime('%H:%M:%S'), drift_status)

## Run an on-demand evaluation

In [ ]:
# Check Drift monitor instance details

monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=drift_monitor_instance_id).result
print(monitor_instance_details)

In [ ]:
# Trigger on-demand run

monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=drift_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id

print(monitoring_run_details)

In [ ]:
# Check run status

drift_run_status = None
while drift_run_status not in ("finished", "error"):
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=drift_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    drift_run_status = monitoring_run_details.entity.status.state
    if drift_run_status not in ("finished", "error"):
        print(datetime.utcnow().strftime("%H:%M:%S"), drift_run_status)
        time.sleep(30)
        
print(datetime.utcnow().strftime("%H:%M:%S"), drift_run_status)

## Display drift metrics

In [ ]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=drift_monitor_instance_id)

<!-- ### Run on-demand Fairness
If you would like to peform an on-demand fairness check, then we need to score a fresh set of data with meta-fields, so that they would be used for indirect bias checking. So the below two cells will score and make sure these records are reached to payload logging table. -->

Author: Ravi Chamarthy (ravi.chamarthy@in.ibm.com)